# Use ColumnTransformer instead of OneHotEncoder

### Based on features from Omnidian database 101, we compare K-Nearest Neighbors, Gradient Boosting, Random Forest, Bagging, and Logistic Regression.

In [1]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import tree
import numpy as np
import pandas as pd
from datetime import datetime
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-paper')

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
import pickle

Get Data

In [ ]:
df101_b = pd.read_csv('data/eda101.csv', parse_dates=['Date_Ticket_Initially_Assigned', 'Date_Ticket_Closed', 'installation_date'])
df101_b.head()

In [ ]:
df101_b.info()

Let's change everything to lowercase to reduce confusions.

In [ ]:
df101_b.rename(str.lower, axis='columns', inplace=True)

In [ ]:
df101_b = df101_b.applymap(lambda s:s.lower() if type(s) == object else s)
df101_b.head()

Both ticket_id and asset_id need to be strings

In [ ]:
df101_b[['ticket_id', 'asset_id']] = df101_b[['ticket_id', 'asset_id']].astype(str)
df101_b.info()

Now drop duplicates

In [ ]:
df101_b.drop_duplicates(inplace=True)

In [ ]:
df101_b.info()

In [ ]:
df101_b.isnull().any()

Assign target to Root_Cause and Train-Test-Split. We'll also take the ticket_id off now so we can use it later to look rows up.

In [ ]:
X = df101_b.drop(['root_cause'], axis=1).copy()
y = df101_b['root_cause']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

Let's try ColumnTransformer. We'll pull ticket_id from the dataframes to keep them from being encoded, then we'll put them back together for our function later.

In [ ]:
train_ticket = X_train.ticket_id

In [ ]:
X_train.drop(['ticket_id'], axis=1, inplace=True)

In [ ]:
test_ticket = X_test.ticket_id

In [ ]:
X_test.drop(['ticket_id'], axis=1, inplace=True)

In [ ]:
# List our categorical features
categoricals = list(X_test.columns[(X_test.dtypes.values == np.dtype('object'))])
categoricals

In [ ]:
X_nums = list(X_test.columns[(X_test.dtypes.values != np.dtype('object'))])
X_nums

In [ ]:
X_train_num = X_train[X_nums].copy()
X_train_num.head()

In [ ]:
X_test_num = X_test[X_nums].copy()
X_test_num.head()

In [ ]:
preprocessor = make_column_transformer( (OneHotEncoder(handle_unknown='ignore'), categoricals))
encoder = preprocessor.fit(X_train)

In [ ]:
X_train_enc = pd.DataFrame(encoder.transform(X_train).toarray(),
                         columns=encoder.get_feature_names())
X_test_enc = pd.DataFrame(encoder.transform(X_test).toarray(),
                        columns=encoder.get_feature_names())

In [ ]:
X_train_enc.head()

In [ ]:
X_train_enc.info()

Now we'll put the ticket_id, numerical columns, and encoded columns all together.

In [ ]:
X_train_num.insert(loc=0, column='ticket_id', value=train_ticket)
X_train_num.head()

In [ ]:
X_train_mega = X_train_num.join(X_train_enc)
X_train_mega.head()

In [ ]:
X_test_num.insert(loc=0, column='ticket_id', value=test_ticket)
X_test_num.head()

In [ ]:
X_test_mega = X_test_num.join(X_test_enc)
X_test_mega.head()

In [ ]:
X_test_mega.isna().any().sum()

In [ ]:
df101_b.iloc[611]

Why did one of our entries go Nan?

In [ ]:
X_train_mega.isna().any().sum()

In [ ]:
df101_b.iloc[689]

In [ ]:
X_train_mega.iloc[689]

In [ ]:
df101_b.loc[df101_b.ticket_id == '19046']

In [ ]:
X_train_mega[X_train_mega.ticket_id == 19046]

In [ ]:
X_train_mega.info()

It's only 1 entry, so we'll drop it for now.

### Modeling

In [ ]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train_enc, y_train)

In [ ]:
gb.score(X_test_enc, y_test)

One hot encoding: since our data is largely categorical but our models require numeric inputs, we one hot encode it.


In [ ]:
# encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
# encoder.fit(X_train)

In [ ]:
# X_train = pd.DataFrame(encoder.transform(X_train).toarray(),
#                          columns=encoder.get_feature_names())
# X_test = pd.DataFrame(encoder.transform(X_test).toarray(),
#                         columns=encoder.get_feature_names())

Training our models.

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
# filename = '101_knn.pkl'
# pickle.dump(knn, open(filename, 'wb'))

In [ ]:
# kn = pickle.load

In [ ]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)

In [ ]:
# filename = '101_lr.pkl'
# pickle.dump(lr, open(filename, 'wb'))

In [ ]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train, y_train)

In [ ]:
# filename = '101_gb.pkl'
# pickle.dump(gb, open(filename, 'wb'))

In [ ]:
dt = DecisionTreeClassifier(random_state=42, min_samples_leaf=30)
dt.fit(X_train, y_train)

In [ ]:
# filename = '101_dt.pkl'
# pickle.dump(dt, open(filename, 'wb'))

In [ ]:
bg = BaggingClassifier(random_state=25565)
bg.fit(X_train, y_train)

In [ ]:
# filename = '101_bg.pkl'
# pickle.dump(bg, open(filename, 'wb'))

In [ ]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train);

In [ ]:
# filename = '101_rf.pkl'
# pickle.dump(rf, open(filename, 'wb'))

Let's show our results

In [ ]:
knn.score(X_test, y_test)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
gb.score(X_test, y_test)

In [ ]:
dt.score(X_test, y_test)

In [ ]:
bg.score(X_test, y_test)

In [ ]:
rf.score(X_test, y_test)

What may be some other classifiers?

In [ ]:
from sklearn.utils.testing import all_estimators

In [ ]:
rf.predict_proba(X_test)[0]

In [ ]:
rf.predict(X_test)[0:5]

In [ ]:
l_props = lr.predict_proba(X_test)[1]

In [ ]:
zipp = list(zip(lr.classes_, l_props))

In [ ]:
#want a dictionary that gives probability for each class, 
#prediction, and ground truth
def display_preds_truth(model, obs, X_test, y_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
#     display['prediction'] = model.predict(X_test)[obs]
    display['ground truth'] = y_test[obs]
    return display
        
    
    

In [ ]:
#want a dictionary that gives probability for each class,
def display_probas(model, obs, X_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
    display['prediction'] = model.predict(X_test)[obs]
    return display

In [ ]:
display_probas(lr, 1, X_test)

In [ ]:
X_test

In [ ]:
y_test